<a href="https://colab.research.google.com/github/IntroComputationalPhysics-UNT/pivot-point-driven-pendulum-henry8675309/blob/main/Stability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [16]:
# create pendulum animation
import matplotlib.pyplot as plt # Import matplotlib.pyplot

def create_pendulum_animation(t, x_pivot, y_pivot, x_lab, y_lab, l=1, show_traj=False, speed_factor=1.0):
    """
    Creates an animation of the pivot-driven pendulum.

    PARAMETERS:
    t : ndarray
        Time array.
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    x_lab : ndarray
        x-positions of the pendulum bob in the lab frame over time.
    y_lab : ndarray
        y-positions of the pendulum bob in the lab frame over time.
    l : float, optional
        pendulum length (default is 1).
    show_traj : bool, optional
        toggle showing the trajectory of the pendulum bob (default is False).
    speed_factor : float, optional
        factor to scale the animation speed (default is 1.0).
        a value > 1.0 slows down the animation, < 1.0 speeds it up.

    RETURNS:
    anim : FuncAnimation
        matplotlib animation object.
    """
    fig, ax = plt.subplots(figsize=(4, 4)) # Increased figure size
    ax.set_xlim(np.min(x_pivot) - l, np.max(x_pivot) + l)
    ax.set_ylim(np.min(y_pivot) - l, np.max(y_pivot) + l)
    # ax.set_aspect('equal', adjustable='box')
    ax.set_aspect('equal')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    # Initialize plot elements
    pivot, = ax.plot([], [], 'o', color='black', markersize=8, label='Pivot')
    pendulum_arm, = ax.plot([], [], '-', color='black', lw=2, label='Pendulum Arm')
    pendulum_bob, = ax.plot([], [], 'o', color='red', markersize=12, label='Pendulum Bob')
    trajectory = None # Initialize trajectory to None

    if show_traj:
        trajectory, = ax.plot([], [], '-', color='gray', lw=1, alpha=0.5, label='Trajectory') # Add trajectory line

    def animate(i):
        # Update the positions of the plot elements
        pivot.set_data([x_pivot[i]], [y_pivot[i]]) # Pass as sequences
        pendulum_arm.set_data([x_pivot[i], x_lab[i]], [y_pivot[i], y_lab[i]])
        pendulum_bob.set_data([x_lab[i]], [y_lab[i]]) # Pass as sequences

        artists = [pivot, pendulum_arm, pendulum_bob] # List of artists to update

        if show_traj and trajectory:
             trajectory.set_data(x_lab[:i+1], y_lab[:i+1]) # Update trajectory data
             artists.append(trajectory) # Add trajectory to the list of artists

        return artists # Return all updated artists

    # Create the animation
    # Adjust the interval based on the average time step in t
    # This aims to make the animation speed consistent with the simulation time
    average_time_step = np.mean(np.diff(t))
    # Scale the interval by the speed_factor
    interval = average_time_step * 1000 * speed_factor # Convert to milliseconds and apply speed_factor.

    anim = FuncAnimation(fig, animate, frames=len(t), interval=interval, blit=True)
    plt.close(fig) # Close the initial figure to prevent it from displaying

    return anim

In [17]:
# animate motion
# plot pendulum angle versus time
# get coordinates for plotting
import numpy as np # Import numpy library
# Define t_min and t_max
t_min = 0 # start time
t_max = 10 # end time
n_points = 300 # number of frames
t_plot = np.linspace(t_min, t_max, n_points)
theta_plot = sol.sol(t_plot)[0] # requires `dense_output=True` in `solve_ivp`
x_pivot, y_pivot = get_pivot_xy(t_plot, omega_d, x_p0, y_p0) # pivot coordinates
x_pendulum, y_pendulum = get_pendulum_xy(t_plot, theta_plot, l) # pendulum coordinates of pendulum (referenced to pivot point)
x_lab, y_lab = get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum) # pendulum coordinates in the lab frame

# animation flags/parameters
show_traj = True # True --> show trajectory; False --> do not show trajectory
speed_factor = 5 # >1 --> slow down animation; <1 --> speed up animation

# create_pendulum_animation
pendulum_animation = create_pendulum_animation(t_plot, x_pivot, y_pivot, x_lab, y_lab, show_traj=show_traj, speed_factor=speed_factor) # create animation object with trajectory shown
HTML(pendulum_animation.to_html5_video()) # display animation

In [14]:
# define functions to get coordinates for plotting
def get_pivot_xy(t, omega_d, x_p0, y_p0):
    """
    Calculates the x and y coordinates of the pivot point over time.

    PARAMETERS:
    t : ndarray
        Time array.
    omega_d : float
        Driving frequency.
    x_p0 : float
        Initial x-position of the pivot.
    y_p0 : float
        Initial y-position of the pivot.

    RETURNS:
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    """
    # Example: Oscillating pivot in x-direction
    x_pivot = x_p0 + 0.5 * np.sin(omega_d * t)  # Example: Add an oscillation amplitude of 0.5
    y_pivot = np.full_like(t, y_p0)  # Example: Keep y-position constant
    return x_pivot, y_pivot

def get_pendulum_xy(t, theta, l):
    """
    Calculates the x and y coordinates of the pendulum bob relative to the pivot.

    PARAMETERS:
    t : ndarray
        Time array.
    theta : ndarray
        Pendulum angle over time (in radians).
    l : float
        Pendulum length.

    RETURNS:
    x_pendulum : ndarray
        x-positions of the pendulum bob relative to the pivot over time.
    y_pendulum : ndarray
        y-positions of the pendulum bob relative to the pivot over time.
    """
    x_pendulum = l * np.sin(theta)
    y_pendulum = -l * np.cos(theta) # Negative because pendulum hangs down
    return x_pendulum, y_pendulum

def get_lab_xy(x_pivot, y_pivot, x_pendulum, y_pendulum):
    """
    Calculates the x and y coordinates of the pendulum bob in the lab frame.

    PARAMETERS:
    x_pivot : ndarray
        x-positions of the pivot over time.
    y_pivot : ndarray
        y-positions of the pivot over time.
    x_pendulum : ndarray
        x-positions of the pendulum bob relative to the pivot over time.
    y_pendulum : ndarray
        y-positions of the pendulum bob relative to the pivot over time.

    RETURNS:
    x_lab : ndarray
        x-positions of the pendulum bob in the lab frame over time.
    y_lab : ndarray
        y-positions of the pendulum bob in the lab frame over time.
    """
    x_lab = x_pivot + x_pendulum
    y_lab = y_pivot + y_pendulum
    return x_lab, y_lab

# Define initial pivot position
x_p0 = 0.0
y_p0 = 0.0

In [12]:
from scipy.integrate import solve_ivp

# Define the differential equation for the pendulum (replace with your actual equation)
def pendulum_ode(t, y, l, omega_d, gamma, A):
    theta, dtheta = y
    # Example equation for a driven damped pendulum
    dydt = [dtheta,
            -(9.81/l) * np.sin(theta) - gamma * dtheta + A * np.cos(omega_d * t)]
    return dydt

# Define initial conditions and parameters (replace with your actual values)
theta0 = np.pi / 2 # initial angle
dtheta0 = 0 # initial angular velocity
l = 1 # pendulum length
omega_d = 1.0 # driving frequency
gamma = 0.1 # damping coefficient
A = 1.0 # driving amplitude

# Solve the differential equation
t_span = [t_min, t_max] # time span for integration
y0 = [theta0, dtheta0] # initial conditions
sol = solve_ivp(pendulum_ode, t_span, y0, dense_output=True, args=(l, omega_d, gamma, A)) # solve with dense output and pass parameters